# Técnicas en aprendizaje estadístico 
## Novozymes Enzyme Stability Prediction
##
### Integrantes
 Andrés Camilo García Moreno
 </br>
 Deyner Elías López Pineda
 </br>
 Daniel Torres Aguirre
 </br>
 Wilmar Andres Garcia Bedoya
 </br>
 Amilder Stewin Ospina Tobon 

In [237]:
#instalacion de la libreria xgboost
! pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [189]:
#librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xboost
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [190]:
#lectura de las bases de datos
data_train = pd.read_csv("train.csv", sep=",")
data_test = pd.read_csv("test.csv", sep=",")
sample = pd.read_csv("sample_submission.csv")
data_train_update = pd.read_csv("train_updates_20220929.csv",index_col="seq_id")

Realizamos la observación de los datos contenidos en el dataset de entrenamiento

In [191]:
data_train.head(n=5)

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


Obtenemos la información de los datos y como están distribuidos

In [192]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            31390 non-null  int64  
 1   protein_sequence  31390 non-null  object 
 2   pH                31104 non-null  float64
 3   data_source       28043 non-null  object 
 4   tm                31390 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB


### Corrección a base de datos

Realizamos la corrección de la base de datos, sugerida por el staff de kagle

In [193]:
nulos = data_train_update.isnull().all("columns")

drop_indices = data_train_update[nulos].index

df_train = data_train.drop(index=drop_indices)

swap_ph_tm_indices = data_train_update[~nulos].index

df_train.loc[swap_ph_tm_indices, ["pH", "tm"]] = data_train_update.loc[swap_ph_tm_indices, ["pH", "tm"]]



contamos la cantidad de aminoácidos presentes en la secuencia, esto corresponde al tamaño de la secuencia
realizamos esta operación para los dataset de entrenamiento y prueba
y este será uno de los datos a usar para la predicción

In [ ]:


data_train['tam_seq'] = [len(val) for val in data_train['protein_sequence']]
data_test['tam_seq'] = [len(val) for val in data_test['protein_sequence']]

cantidad de nulos

In [194]:

data_train.isnull().sum()

seq_id                 0
protein_sequence       0
pH                   286
data_source         3347
tm                     0
tam_seq                0
dtype: int64

Datos de entrenamiento con la variable tam_seq

In [196]:

data_train.head()

,seq_id,protein_sequence,pH,data_source,tm,tam_seq
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,341
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,286
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,497
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,265
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,1451


eliminamos la variable data_source, ya que esta solo nos indica de donde fue tomada
la información y esto no nos sirve para realizar la predicción.

In [197]:

data_train = data_train.drop(["data_source"],axis=1)
data_test = data_test.drop(["data_source"],axis=1)

Contamos la cantidad de valores nulos

In [198]:

data_train.isnull().sum()

seq_id                0
protein_sequence      0
pH                  286
tm                    0
tam_seq               0
dtype: int64

Reemplazamos los valores nulos presentes en la columna pH por el promedio de esta.

In [199]:
data_train["pH"].fillna(data_train["pH"].mean(), inplace=True)

Verificamos que no queden nulos presentes

In [200]:
data_train.isnull().sum()

seq_id              0
protein_sequence    0
pH                  0
tm                  0
tam_seq             0
dtype: int64

verificamos la información contenida en el dataset de test

In [201]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2413 entries, 0 to 2412
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   seq_id            2413 non-null   int64 
 1   protein_sequence  2413 non-null   object
 2   pH                2413 non-null   int64 
 3   tam_seq           2413 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 75.5+ KB


definimos los datos de entrenamiento y la variable objetivo

In [203]:
entrenamiento = data_train.drop(["tm","protein_sequence","seq_id"],axis=1)
varObj = data_train["tm"]


dividimos el dataset en un 80% datos de entrenamiento y un 20% datos de prueba


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(entrenamiento,varObj,test_size=.2,random_state=123)

definimos el modelo a usar
</br>
utilizaremos XGBOOST, Este algoritmo se caracteriza por obtener buenos resultados de predicción 
con relativamente poco esfuerzo, en muchos casos equiparables o mejores que los devueltos por modelos
más complejos computacionalmente, en particular para problemas con datos heterogéneos.

In [220]:
model = xboost.XGBRegressor()
model.fit(X_train, y_train)


Realizamos una predicción para los datos de prueba


In [ ]:
y_predic = model.predict(X_test)

calculamos el Error absoluto medio

In [221]:
print('Mean Absolute Error =', mean_absolute_error(y_test, y_predic))

Mean Absolute Error = 8.328136288950724


Realizamos la prediccion con el dataset proporcionado en la competencia

In [230]:
data_prediction = data_test.drop(["seq_id","protein_sequence"],axis=1)
prediccion = model.predict(data_prediction)

Creamos el nuevo dataset con la entrega a realizar

In [232]:
entrega = sample.copy()
entrega["tm"] = prediccion
entrega

,seq_id,tm
0,31390,61.583820
1,31391,61.583820
2,31392,57.713017
3,31393,61.583820
4,31394,61.583820
...,...,...
2408,33798,61.583820
2409,33799,61.583820
2410,33800,61.583820
2411,33801,61.583820


Exportamos al formato csv

In [236]:
entrega.to_csv("submission.csv",index=False)

### Referencias

(Tutorial xgboost)[https://medium.com/@jboscomendoza/tutorial-xgboost-en-python-53e48fc58f73]
</br>
(Staff Kaggle)[https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction/discussion/356251]